In [0]:
-- Databricks DLT Pipeline - Gold Layer Dimensional Model
-- File: gold_teams_dimensional.sql

-- Current Team Dimension (Type 1 SCD)
CREATE OR REFRESH MATERIALIZED VIEW ${catalog}.ncaa_gold.dim_team (
  CONSTRAINT unique_teams EXPECT (team_id IS NOT NULL),
  CONSTRAINT recent_data EXPECT (data_year >= YEAR(CURRENT_DATE()) - 2),
  CONSTRAINT has_conference EXPECT (conference_name IS NOT NULL),
  CONSTRAINT complete_team_info EXPECT (
    school_name IS NOT NULL 
    AND team_abbreviation IS NOT NULL 
    AND conference_name IS NOT NULL
  )
)
COMMENT "Gold layer - Current team dimension (Type 1 SCD) for operational analytics"
TBLPROPERTIES (
  "delta.autoOptimize.optimizeWrite" = "true",
  "delta.autoOptimize.autoCompact" = "true",
  "pipelines.autoOptimize.managed" = "true"
)
AS (
  WITH latest_teams AS (
    SELECT 
      *,
      ROW_NUMBER() OVER (
        PARTITION BY team_id 
        ORDER BY data_year DESC, silver_processed_at DESC
      ) as row_number
    FROM ${catalog}.ncaa_silver.teams_cleaned
    WHERE team_id IS NOT NULL 
      AND school_name IS NOT NULL
  )
  SELECT 
    team_id,
    school_name,
    team_abbreviation,
    team_mascot,
    conference_name,
    division_name,
    venue_id,
    venue_name,
    venue_city,
    venue_state,
    venue_zip,
    venue_capacity,
    venue_construction_year,
    venue_latitude,
    venue_longitude,
    is_dome,
    has_grass_field,
    venue_elevation,
    venue_timezone,
    country_code,
    primary_logo_url,
    twitter_handle,
    data_year,
    venue_age,
    capacity_tier,
    geographic_region,
    data_completeness_score,
    CASE WHEN data_year = YEAR(CURRENT_DATE()) THEN true ELSE false END as is_current_season,
    YEAR(CURRENT_DATE()) - data_year as years_since_update,
    CONCAT(school_name, ' ', team_mascot) as team_display_name,
    CASE 
      WHEN division_name IS NOT NULL THEN CONCAT(conference_name, ' ', division_name)
      ELSE conference_name 
    END as conference_division_full,
    CURRENT_TIMESTAMP() as dim_last_updated
  FROM latest_teams
  WHERE row_number = 1
);

-- Historical Team Dimension (SCD Type 2) Target Table
CREATE OR REFRESH STREAMING TABLE ${catalog}.ncaa_gold.dim_team_scd (
  CONSTRAINT scd_valid_team_id EXPECT (team_id IS NOT NULL),
  CONSTRAINT scd_has_school EXPECT (school_name IS NOT NULL),
  CONSTRAINT scd_has_conference EXPECT (conference_name IS NOT NULL),
  CONSTRAINT scd_reasonable_year EXPECT (data_year >= 2000 AND data_year <= YEAR(CURRENT_DATE()) + 1)
)
COMMENT "Gold layer - Team historical dimension (Type 2 SCD) tracking conference realignment and changes over time"
TBLPROPERTIES (
  "delta.autoOptimize.optimizeWrite" = "true",
  "delta.autoOptimize.autoCompact" = "true",
  "pipelines.autoOptimize.managed" = "true"
);

-- APPLY CHANGES INTO for Automatic SCD Type 2
APPLY CHANGES INTO ${catalog}.ncaa_gold.dim_team_scd
FROM STREAM(${catalog}.ncaa_silver.teams_cleaned)
KEYS (team_id)
SEQUENCE BY data_year
COLUMNS * EXCEPT (silver_processed_at)
STORED AS SCD TYPE 2;


# 🚀 Gold Layer - Teams Dimensional Model

## 📊 Overview
This documentation supports the **gold_teams_dimensional.sql** DLT pipeline script.

### Purpose
- **`dim_team`**: Current state dimension (Type 1 SCD) for operational analytics
- **`dim_team_scd`**: Historical dimension (Type 2 SCD) for conference realignment tracking

---

## ⚡ APPLY CHANGES INTO - Latest DLT Feature

### 🎯 Proper SQL Syntax
```sql
CREATE OR REFRESH STREAMING TABLE target_table;

APPLY CHANGES INTO target_table
FROM STREAM(source_table)
KEYS (key_column)
SEQUENCE BY sequence_column
COLUMNS * EXCEPT (excluded_columns)
STORED AS SCD TYPE 2;
```

### 🏆 Key Benefits
- ✅ **Zero SCD boilerplate code** - DLT handles all SCD logic automatically
- 🚀 **Automatic change detection** - tracks conference, venue, mascot changes
- ⚡ **Built-in columns**: `__start_at`, `__end_at`, `__is_current` 
- 🛠️ **Late-arriving data handling**
- 📈 **Performance optimized**
- 🎉 **Handles deletions and updates automatically**

---

## 🔍 Built-in DLT Data Quality Expectations

### SQL Expectation Syntax
```sql
CREATE OR REFRESH MATERIALIZED VIEW table_name (
  CONSTRAINT expectation_name EXPECT (condition)
)
```

### 📊 Gold Layer Expectations
- **`unique_teams`**: Ensures all teams have valid IDs
- **`recent_data`**: Validates data is within 2 years 
- **`has_conference`**: Tracks conference data completeness
- **`complete_team_info`**: Ensures core team fields are populated

### 🎯 SCD Table Expectations
- **`scd_valid_team_id`**: Critical for SCD key integrity
- **`scd_has_school`**: Ensures meaningful team records
- **`scd_has_conference`**: Tracks conference data over time
- **`scd_reasonable_year`**: Validates historical data ranges

### ✅ Built-in DLT Features
- All use **EXPECT** (retain records, track metrics in DLT UI)
- Monitor in **"Data quality"** tab for pass/fail rates
- Same expectation framework as bronze/silver layers
- Automatic metric collection and reporting

---

## 🏈 Automatic SCD Type 2 Features

DLT automatically adds these columns to `dim_team_scd`:
- **`__start_at`**: TIMESTAMP when this version became effective
- **`__end_at`**: TIMESTAMP when this version expired (null = current)
- **`__is_current`**: BOOLEAN flag for current record

---

## 🎯 Conference Realignment Demo

### Show automatic SCD tracking for Texas
```sql
SELECT 
  school_name,
  conference_name,
  __start_at,
  __end_at,
  __is_current
FROM ${catalog}.ncaa_gold.dim_team_scd 
WHERE school_name = 'Texas'
ORDER BY __start_at;
```

**Result automatically shows:**
```
Texas | Big 12 | 2023-07-01 | 2024-06-30 | false
Texas | SEC    | 2024-07-01 | null       | true
```

---

## 🔗 Star Schema Relationships

### Current state joins (90% of queries)
```sql
SELECT p.*, t.school_name, t.conference_name
FROM fact_play_by_play p
JOIN ${catalog}.ncaa_gold.dim_team t ON p.team_id = t.team_id;
```

### Historical point-in-time joins (conference realignment analysis)
```sql
SELECT p.*, t.school_name, t.conference_name
FROM fact_play_by_play p
JOIN ${catalog}.ncaa_gold.dim_team_scd t ON p.team_id = t.team_id 
  AND p.game_date >= t.__start_at 
  AND (p.game_date < t.__end_at OR t.__end_at IS NULL);
```

---

## 📈 Demo Script for Latest Features

### 🚀 Conference Changes Automatically Tracked
```sql
-- Teams that changed conferences
SELECT DISTINCT team_id, school_name, COUNT(*) as conference_changes
FROM ${catalog}.ncaa_gold.dim_team_scd
GROUP BY team_id, school_name
HAVING COUNT(*) > 1
ORDER BY conference_changes DESC;
```

### 🎯 Point-in-Time Conference Membership
```sql
-- Big 12 teams on specific date
SELECT DISTINCT school_name
FROM ${catalog}.ncaa_gold.dim_team_scd
WHERE conference_name = 'Big 12'
  AND '2023-12-01' >= __start_at 
  AND ('2023-12-01' < __end_at OR __end_at IS NULL);
```

### 📊 Conference Realignment Timeline
```sql
SELECT 
  school_name,
  conference_name,
  __start_at as conference_start,
  __end_at as conference_end,
  CASE WHEN __is_current THEN 'Current' ELSE 'Historical' END as status
FROM ${catalog}.ncaa_gold.dim_team_scd
WHERE school_name IN ('Texas', 'Oklahoma', 'USC', 'UCLA')
ORDER BY school_name, __start_at;
```

---

## 🔥 Why APPLY CHANGES INTO is Revolutionary

### ❌ Traditional SCD (100+ lines of code)
- Complex MERGE statements
- Manual change detection logic
- Custom `__start_at`/`__end_at` handling
- Error-prone edge cases
- Performance tuning required

### ✅ APPLY CHANGES INTO (2 lines of code)
- 🚀 Declarative syntax
- ⚡ Automatic change detection  
- 🛠️ Built-in SCD management
- 🎉 Zero maintenance required
- 📈 Performance optimized by default

**Perfect for demos showing how Databricks simplifies complex data engineering!** 🎊

---

## 🏗️ Pipeline Execution Flow
1. **Bronze**: `teams_raw` (streaming table) - Raw JSON ingestion
2. **Silver**: `teams_cleaned` (streaming table) - Cleaned and flattened *(required for APPLY CHANGES INTO)*
3. **Gold**: `dim_team` (materialized view) - Current state dimension
4. **Gold**: `dim_team_scd` (APPLY CHANGES INTO) - Historical SCD Type 2

**Note**: Silver layer must be streaming table for APPLY CHANGES INTO to work in gold layer.

---

## 📋 Tables Created
- **`${catalog}.ncaa_gold.dim_team`** - Current teams for operational analytics
- **`${catalog}.ncaa_gold.dim_team_scd`** - Historical teams with automatic SCD Type 2

---

## 📊 Monitoring and Data Quality
- Use DLT UI **"Data quality"** tab to monitor all expectations
- Track conference realignment changes automatically
- Monitor data completeness across all team attributes
- Built-in performance optimization and auto-compaction

---

## ⚙️ Pipeline Configuration
Set catalog parameter in DLT pipeline: `{"catalog": "dfl_dev"}`
- Both tables refresh when silver teams_cleaned changes
- APPLY CHANGES INTO automatically processes incremental changes
